In [18]:
import geopandas as gpd
import pandas as pd
urban_villages = gpd.read_file('data/urban_villages.shp')
cur_plan = pd.read_csv('Plans/inferred_plan.csv')
urban_villages

,assign_row,assign_col,geometry
0,33.0,39.0,"POLYGON ((12973408.791 4848757.915, 12973430.7..."
1,31.0,33.0,"POLYGON ((12965239.832 4851295.885, 12965274.9..."
2,27.0,27.0,"POLYGON ((12957516.525 4856968.773, 12957511.1..."
3,34.0,27.0,"POLYGON ((12957066.472 4847557.453, 12957005.8..."
4,40.0,36.0,"MULTIPOLYGON (((12968967.344 4840313.774, 1296..."
...,...,...,...
939,26.0,14.0,"POLYGON ((12939653.615 4858756.780, 12939858.2..."
940,21.0,18.0,"POLYGON ((12945819.822 4865293.534, 12945704.6..."
941,31.0,35.0,"POLYGON ((12967597.895 4851467.967, 12967598.2..."
942,21.0,32.0,"POLYGON ((12963156.515 4864347.589, 12963156.9..."


In [19]:

def generate_plan_gdf(plan, gdf):
    """
    Generates a GeoDataFrame containing geometries from `gdf` whose assigned grid appears in the `plan`.

    Args:
    - plan (pd.DataFrame): A DataFrame containing the plan with columns:
                           ['year', 'row', 'column', 'r_r', 'r_c', 'r_poi', 'FAR'].
    - gdf (gpd.GeoDataFrame): A GeoDataFrame with columns:
                              ['geometry', 'assigned_grid'], where 'assigned_grid' is a tuple (row, column).

    Returns:
    - gpd.GeoDataFrame: A GeoDataFrame with geometries from the plan, including columns:
                        ['geometry', 'year', 'row', 'column', 'r_r', 'r_c', 'r_poi', 'FAR'].
    """
    # Merge plan and gdf based on the grid coordinates
    plan['assigned_grid'] = list(zip(plan['row'], plan['column']))
    gdf['assigned_grid'] = list(zip(gdf['assign_row'], gdf['assign_col']))
    merged = gdf.merge(plan, on='assigned_grid', how='inner')

    # Select relevant columns
    output_columns = ['geometry', 'year', 'row', 'column', 'r_r', 'r_c', 'r_poi', 'FAR']
    result_gdf = merged[output_columns]

    return gpd.GeoDataFrame(result_gdf, geometry='geometry')

result = generate_plan_gdf(cur_plan, urban_villages)

In [20]:
result

,geometry,year,row,column,r_r,r_c,r_poi,FAR
0,"POLYGON ((12957516.525 4856968.773, 12957511.1...",6,27,27,0.2,0.7,0.1,2.2
1,"POLYGON ((12957066.472 4847557.453, 12957005.8...",2,34,27,0.1,0.6,0.3,2.2
2,"MULTIPOLYGON (((12968967.344 4840313.774, 1296...",4,40,36,0.6,0.2,0.2,2.0
3,"POLYGON ((12968616.010 4843257.164, 12968583.4...",7,38,36,0.3,0.6,0.1,2.2
4,"MULTIPOLYGON (((12934731.858 4840938.869, 1293...",2,39,10,0.1,0.6,0.3,2.1
...,...,...,...,...,...,...,...,...
562,"POLYGON ((12980305.635 4885611.371, 12980305.2...",3,5,45,0.2,0.4,0.4,1.8
563,"MULTIPOLYGON (((12945444.136 4887038.821, 1294...",2,4,18,0.3,0.6,0.1,2.0
564,"POLYGON ((12969481.934 4857051.217, 12969401.4...",6,27,36,0.2,0.5,0.3,2.2
565,"POLYGON ((12945819.822 4865293.534, 12945704.6...",9,21,18,0.2,0.6,0.2,1.9


In [22]:
result.to_file('Plans/plan.shp')